<a href="https://colab.research.google.com/github/singhsmita/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# Find the latest version of spark 3.3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.2'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [993 kB]
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-04-20 22:11:39--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.25MB/s    in 0.2s    

2023-04-20 22:11:40 (6.25 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Shoes_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   18069663|R3P2HIOQCIN5ZU|B000XB31C0|     265024781|Minnetonka Men's ...|           Shoes|          1|            0|          0|   N|                Y|                   .|Do not buy: reall...|2015-08-31 00:00:00|
|         US|   16251825|R12VVR0WH5Q24V|B00CFYZH5W|     259035853|Teva Men's Pajaro...| 

In [6]:
vine_df = df.select(["review_id" ,"star_rating" ,"helpful_votes" ,"total_votes" ,"vine" ,"verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3P2HIOQCIN5ZU|          1|            0|          0|   N|                Y|
|R12VVR0WH5Q24V|          5|            0|          0|   N|                Y|
| RNCCKB6TV5EEF|          4|            0|          0|   N|                Y|
|R2NZXYIVCGB13W|          5|            0|          6|   N|                Y|
|R2EQ1TG9IT3OEQ|          3|            0|          0|   N|                Y|
|R1WXA9JSC2H1U1|          5|            1|          1|   N|                Y|
|R12ENYLFGGNWRV|          5|            1|          1|   N|                Y|
|R2R07E5PNXEUO5|          4|            0|          0|   N|                Y|
|R27BA52AKWMWN3|          5|            0|          0|   N|                Y|
| RLF8DOID2KD5O|          3|            0|          0|   N|     

In [10]:
# Step 1: Create a new DF: condition => total_votes>=20
filtered_vine_df = vine_df.filter("total_votes>=20")
filtered_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R37F42INKX7L9K|          5|           45|         49|   N|                Y|
|R2EHKYNEP8WVSR|          5|           25|         25|   N|                Y|
| RXOS7BHID0UHL|          5|           16|         27|   N|                N|
|R33HHGFPB403GM|          5|           19|         21|   N|                Y|
| RY9O9XNLP464N|          2|           19|         22|   N|                Y|
|R2VP11C28JAEZF|          5|           30|         30|   N|                Y|
|R1TXGR1HA2M3P3|          5|           28|         29|   N|                Y|
| R6OD85TMEMHQQ|          5|           28|         28|   N|                Y|
|R1G4PAJXP3FTN7|          2|           43|         51|   N|                Y|
|R2P2S8UGUMCOLX|          5|           21|         22|   N|     

In [11]:
# Step 2: Create a new DF: condition => helpful_votes/total_votes >= 50%
#helpful_votes_df = filtered_vine_df.withColumn('greaterthan50', filtered_vine_df['helpful_votes'] / filtered_vine_df['total_votes']).filter('greaterthan50>=0.5')
helpful_votes_df = filtered_vine_df.filter("(helpful_votes/total_votes)>=0.5")
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R37F42INKX7L9K|          5|           45|         49|   N|                Y|
|R2EHKYNEP8WVSR|          5|           25|         25|   N|                Y|
| RXOS7BHID0UHL|          5|           16|         27|   N|                N|
|R33HHGFPB403GM|          5|           19|         21|   N|                Y|
| RY9O9XNLP464N|          2|           19|         22|   N|                Y|
|R2VP11C28JAEZF|          5|           30|         30|   N|                Y|
|R1TXGR1HA2M3P3|          5|           28|         29|   N|                Y|
| R6OD85TMEMHQQ|          5|           28|         28|   N|                Y|
|R1G4PAJXP3FTN7|          2|           43|         51|   N|                Y|
|R2P2S8UGUMCOLX|          5|           21|         22|   N|     

In [22]:
# Step 3: Create a new DF where review was written => vine=='Y'
vine_review_df = helpful_votes_df.filter(helpful_votes_df['vine'] == 'Y')
vine_review_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2N45ZKRRZS856|          5|           21|         22|   Y|                N|
| R5OMLMK13A8NS|          5|           34|         38|   Y|                N|
|R2MPEQ4SPTEQNS|          4|          180|        184|   Y|                N|
| RIR0D3KJ0CQ31|          4|           21|         21|   Y|                N|
|R1SPWJDHUWWC5E|          5|           88|         98|   Y|                N|
|R1X6M5XA3FT98W|          5|           24|         26|   Y|                N|
|R37VCW6HA0Z72T|          5|           27|         28|   Y|                N|
|R2XRYNV2SY3ZKL|          5|           53|         56|   Y|                N|
|R1Y93KWKAX1P5N|          2|           26|         31|   Y|                N|
|R2QVTDYYLTP8SL|          5|           21|         24|   Y|     

In [30]:
# Step 4: Create a new DF where review was not written => vine=='N'

vine_NoReview_df = helpful_votes_df.filter(helpful_votes_df['vine'] == 'N')
vine_NoReview_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R37F42INKX7L9K|          5|           45|         49|   N|                Y|
|R2EHKYNEP8WVSR|          5|           25|         25|   N|                Y|
| RXOS7BHID0UHL|          5|           16|         27|   N|                N|
|R33HHGFPB403GM|          5|           19|         21|   N|                Y|
| RY9O9XNLP464N|          2|           19|         22|   N|                Y|
|R2VP11C28JAEZF|          5|           30|         30|   N|                Y|
|R1TXGR1HA2M3P3|          5|           28|         29|   N|                Y|
| R6OD85TMEMHQQ|          5|           28|         28|   N|                Y|
|R1G4PAJXP3FTN7|          2|           43|         51|   N|                Y|
|R2P2S8UGUMCOLX|          5|           21|         22|   N|     

**Step 5 : Total number of Reviews , the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid)**

In [31]:

# Step 5.1.a: total number of reviews
Vine_Total_df = helpful_votes_df.agg({'review_id':'count'}).withColumnRenamed("count(review_id)", "Total_Vine_Reviews")
Vine_Total_df.show()

+------------------+
|Total_Vine_Reviews|
+------------------+
|             27009|
+------------------+



In [37]:
# Step 5.1.b: # Determine the total number of reviews - PAID
Vine_Paid_reviews = vine_review_df.agg({'review_id':'count'}).withColumnRenamed("count(review_id)", "Total_Paid_Reviews")
Vine_Paid_reviews.show()

+------------------+
|Total_Paid_Reviews|
+------------------+
|                22|
+------------------+



In [32]:
# Determine the total number of reviews - UNPAID
Vine_Unpaid_reviews = vine_NoReview_df.agg({'review_id':'count'}).withColumnRenamed("count(review_id)", "Total_Unpaid_Reviews")
Vine_Unpaid_reviews.show()


+--------------------+
|Total_Unpaid_Reviews|
+--------------------+
|               26987|
+--------------------+



**Total Number of 5-star reviews**

In [40]:
# Determine Total no of 5 star reviews
Total_5Star_reviews = helpful_votes_df.filter("star_rating == 5").agg({'star_rating':'count'}).withColumnRenamed("count(star_rating)", "Total_5Star_Rating")
Total_5Star_reviews.show()

+------------------+
|Total_5Star_Rating|
+------------------+
|             14488|
+------------------+



In [34]:
# Determine the number of 5-star reviews - PAID
paid_5star_reviews = vine_review_df.filter('star_rating==5').agg({'star_rating':'count'}).withColumnRenamed("count(star_rating)", "5-stars_Paid_Review")
paid_5star_reviews.show()

+-------------------+
|5-stars_Paid_Review|
+-------------------+
|                 13|
+-------------------+



In [35]:

# Determine the number of 5-star reviews - UNPAID
unpaid_5star_reviews = vine_NoReview_df.filter('star_rating==5').agg({'star_rating':'count'}).withColumnRenamed("count(star_rating)", "5-stars_unPaid_Review")
unpaid_5star_reviews.show()

+---------------------+
|5-stars_unPaid_Review|
+---------------------+
|                14475|
+---------------------+



**Total Percentage of 5-star reviews**

In [44]:
# Paid reviews as % total 5-star reviews

Paid_reviews_percentage = paid_5star_reviews.collect()[0]['5-stars_Paid_Review'] / Total_5Star_reviews.collect()[0]['Total_5Star_Rating'] * 100
round(Paid_reviews_percentage,2)

0.09

In [43]:
# Determine the percentage of 5-star reviews as part of total vine- PAID

Paid_reviews_percentage = paid_5star_reviews.collect()[0]['5-stars_Paid_Review'] / Vine_Paid_reviews.collect()[0]['Total_Paid_Reviews'] * 100
round(Paid_reviews_percentage,2)

59.09

In [42]:
# Unpaid reviews as % total 5-star reviews

Unpaid_reviews_percentage  = unpaid_5star_reviews.collect()[0]['5-stars_unPaid_Review'] / Total_5Star_reviews.collect()[0]['Total_5Star_Rating'] * 100
round(Unpaid_reviews_percentage,2)

99.91

In [45]:
# Determine the percentage of 5-star reviews part of total vine - UNPAID

Paid_reviews_percentage = unpaid_5star_reviews.collect()[0]['5-stars_unPaid_Review'] / Vine_Unpaid_reviews.collect()[0]['Total_Unpaid_Reviews'] * 100
round(Paid_reviews_percentage,2)

53.64